In [1]:
!pip install groq python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 6.3 MB/s eta 0:00:00


In [2]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API Key: ")

Enter your Groq API Key: ··········


In [13]:
import os
from groq import Groq

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# ==============================
# 1️⃣ DEFINE EXPERT CONFIGURATION
# ==============================

MODEL_NAME = "llama-3.1-8b-instant"

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a Senior Technical Support Engineer.
Be precise, logical, and code-focused.
Explain errors clearly and provide corrected code when needed.
Avoid unnecessary fluff."""
    },
    "billing": {
        "system_prompt": """You are a Customer Billing Specialist.
Be empathetic and professional.
Clearly explain charges, refunds, and policies.
Reassure the customer and guide them on next steps."""
    },
    "general": {
        "system_prompt": """You are a helpful and friendly assistant.
Respond naturally and conversationally."""
    }
}

In [14]:
def route_prompt(user_input):
    routing_prompt = f"""
Classify the following user query into one of these categories:
[technical, billing, general]

Return ONLY the category name. Do not explain.

Query:
{user_input}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": "You are a strict classifier."},
            {"role": "user", "content": routing_prompt}
        ],
        temperature=0
    )

    category = response.choices[0].message.content.strip().lower()

    # Safety fallback
    if category not in MODEL_CONFIG:
        category = "general"

    return category

In [15]:
def process_request(user_input):

    # Step 1: Route the query
    category = route_prompt(user_input)
    print(f"🔎 Routed to: {category.upper()} expert")

    # Step 2: Get system prompt
    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    # Step 3: Call expert model
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.7
    )

    return response.choices[0].message.content

In [16]:
print(process_request("My Python script is throwing an IndexError on line 5."))

🔎 Routed to: TECHNICAL expert
To assist you, I'll need a bit more information:

1. What is the line of code that's causing the error (line 5)?
2. What's the error message?
3. Can you provide the surrounding code (lines 3-6) that's relevant to the error?
4. Are you working with a specific dataset or file?

Please provide the necessary details, and I'll do my best to help you troubleshoot the issue.

Once I have this information, I'll provide a clear explanation of the error and, if necessary, suggest corrected code to resolve the issue. 

Assuming a basic scenario, if you're trying to access an element in a list that doesn't exist, the error message should look something like this:

`IndexError: list index out of range`

This usually means that your list is empty or you're trying to access an index that's larger than the list's length.


In [17]:
print(process_request("I was charged twice for my subscription this month."))

🔎 Routed to: BILLING expert
I'm so sorry to hear that you were charged twice for your subscription this month. I understand how frustrating that must be for you.

Let me see if I can help resolve this issue for you right away. Can you please confirm your subscription details, such as your name, email address associated with your subscription, and the date you noticed the duplicate charges? This information will help me locate the issue and process a refund for the duplicate charge.

Also, I'd like to assure you that we take situations like this very seriously and want to make it right. We're committed to providing fair and honest billing practices.

Once I've located the issue, I'll provide you with an update on what we can do to rectify the situation and prevent it from happening again in the future. Would you like me to proceed with checking on the duplicate charges?


In [18]:
print(process_request("Tell me a fun fact about space."))

🔎 Routed to: GENERAL expert
I've got a fun one for you. Did you know that there's a giant storm on Jupiter that's been raging for at least 187 years? It's called the Great Red Spot, and it's a massive anticyclonic storm that's so large it could swallow several Earths whole. It's been continuously observed since 1831, and scientists are still amazed by its sheer scale and longevity. Isn't that mind-blowing?


In [19]:
MODEL_CONFIG["tool"] = {
    "system_prompt": "You are a financial data tool."
}